# Лабораторная на Классификацию ||

В ходе этой работы мы проведём классификацию на реальных данных при помощи логистической регрессии

## Импортируем библиотеки

In [27]:
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score

## Получим данные и сразу их предобработаем (данные не отличаются от блокнота 1_2 - тот же набор, соответственно весь анализ был уже проведён)

In [2]:
df = pd.read_csv('./data/bank-additional-full.csv', sep=';')

# Разделим данные на категориальные и числовые
numerical_cols = ['age', 'duration', 'campaign', 'pdays', 'previous',
                  'emp.var.rate', 'cons.price.idx', 'cons.conf.idx',
                  'euribor3m', 'nr.employed']
categorical_cols = ['job', 'marital', 'education', 'default', 'housing',
                    'loan', 'contact', 'month', 'day_of_week', 'poutcome']

# Обработаем пропуски и заменим 'unknown' на наиболее частое значение
for col in categorical_cols:
    if (df[col] == 'unknown').sum() > 0:
        mode_value = df[col][df[col] != 'unknown'].mode()[0]
        df[col] = df[col].replace('unknown', mode_value)

# Проведём One-Hot кодирование для категориальных признаков
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Стандартизируем числовые признаки
scaler = StandardScaler()
df_encoded[numerical_cols] = scaler.fit_transform(df_encoded[numerical_cols])

## Обучим базовую модель логистической регрессии

In [19]:
# Разделение на признаки и целевую переменную
X_numerical = df_encoded[numerical_cols]
y = df['y']

# Разделим на train/test
X_train, X_test, Y_train, Y_test = train_test_split(
    X_numerical, y, test_size=0.2, random_state=42, stratify=y
)

# Создадим baseline логистической регрессии на числовых признаках
baseline_model = LogisticRegression(
    random_state=42,
    max_iter=2000,
    class_weight='balanced'
)
baseline_model.fit(X_train, Y_train)

Y_pred = baseline_model.predict(X_test)

print(classification_report(Y_test, Y_pred))


              precision    recall  f1-score   support

          no       0.98      0.85      0.91      7310
         yes       0.43      0.89      0.58       928

    accuracy                           0.85      8238
   macro avg       0.71      0.87      0.75      8238
weighted avg       0.92      0.85      0.87      8238



У нас вышел сильный дисбаланс классов (отказов куда больше, нежели положительных результатов), а так же низкая точность для "yes" - когда модель предсказывать "yes", то ошибается БОЛЕЕ ЧЕМ НА 50% (это критично) (модель жертвует точностью ради полноты). Но в целом после обучения на числовых признаках мы получили неплохие результаты, теперь можно улучшать модельку дальше (в частности разобраться с низким precision для "yes")

## Улучшим качество модели

Давайте попробуем убрать балансировку и поработаем с некоторыми категориальными признаками

In [20]:
# Проверим разные веса и выберем подходящий
for weight_yes in [1, 2, 3, 4, 5, 6]:
    new_baseline_model = LogisticRegression(
        random_state=42,
        max_iter=2000,
        class_weight={'no': 1, 'yes': weight_yes}
    )
    new_baseline_model.fit(X_train, Y_train)

    y_pred = new_baseline_model.predict(X_test)
    print(f"Weight yes = {weight_yes}")
    print(classification_report(Y_test, y_pred))
    print("------------------------------------------------------")

Weight yes = 1
              precision    recall  f1-score   support

          no       0.93      0.98      0.95      7310
         yes       0.69      0.39      0.50       928

    accuracy                           0.91      8238
   macro avg       0.81      0.68      0.72      8238
weighted avg       0.90      0.91      0.90      8238

------------------------------------------------------
Weight yes = 2
              precision    recall  f1-score   support

          no       0.95      0.95      0.95      7310
         yes       0.61      0.58      0.59       928

    accuracy                           0.91      8238
   macro avg       0.78      0.77      0.77      8238
weighted avg       0.91      0.91      0.91      8238

------------------------------------------------------
Weight yes = 3
              precision    recall  f1-score   support

          no       0.96      0.93      0.95      7310
         yes       0.56      0.69      0.62       928

    accuracy               

По метрикам разных весов мы видим, что как будто бы модель при 3 ведёт себя интереснее, нежели другие и базовая модель

In [24]:
# Добавим категориальные признаки, обучим модель и посмотрим что вышло
X_full = df_encoded.drop('y', axis=1)
y = df['y']
X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(
    X_full, y, test_size=0.2, random_state=42, stratify=y
)

model_full = LogisticRegression(
    random_state=42,
    max_iter=2000,
    class_weight={'no': 1, 'yes': 3}
)
model_full.fit(X_train_f, y_train_f)

y_pred_f = model_full.predict(X_test_f)
print(classification_report(y_test_f, y_pred_f))

              precision    recall  f1-score   support

          no       0.97      0.93      0.95      7310
         yes       0.57      0.74      0.65       928

    accuracy                           0.91      8238
   macro avg       0.77      0.84      0.80      8238
weighted avg       0.92      0.91      0.91      8238



При использовании категориальных признаков наша модель стала "умнее" по одним метрикам, либо не стала "тупее" по другим - это очень хорошо (не ушли в минус, значит улучшили). Можно было бы более умно обработать признаки, провести Future engineering, посмотреть корреляции признаков и это использовать, а так же углубиться в домен и понять что лучше переиспользовать, но здесь проводить это не будем (ТЗ попытаться улучшить выполнено (даже выполнено с отличием, тк мы попытались и у нас вышло)).

## Вывод (серединный)
В ходе работы мы обучили модельку на числовых признаках, немного поигрались с параметрами модельки и выбрали лучший, а после дообучили (переучили) на категориальных признаках (использовав только One-Hot кодирование). В целом вышла модель выше 50% по всем метрикам, но опять же проблемы с yes - моделька слишком плохо работает с этим признаком и надо что-то с этим делать (мы попытались и даже улучшили, но нужно применять более умные вещи, описанные выше). Считаю первую работу (выполненную на реальных данных) на классификацию вполне успешной.

## Давайте теперь поработаем с другой моделью и сравним метрики

Выберем из 3 предложенных моделей (ближайшие соседи, деревья решений, метод опорных векторов) лучшую. Этой моделью станет Decision Tree, тк она будет лучше работать с дисбалансом, нежели остальные модели (идеальным вариантом тут бы был случайный лес, но его запретили юзать - грусть печаль тоска апатия слёзы и депрессия в 0 лет)

Создадим базовое дерево, обучим на всех данных и посмотрим метрики

In [26]:
base_tree = DecisionTreeClassifier(random_state=42, class_weight='balanced')
base_tree.fit(X_train_f, y_train_f)

y_pred_tree = base_tree.predict(X_test_f)

print(classification_report(y_test_f, y_pred_tree))

              precision    recall  f1-score   support

          no       0.94      0.94      0.94      7310
         yes       0.55      0.54      0.54       928

    accuracy                           0.90      8238
   macro avg       0.74      0.74      0.74      8238
weighted avg       0.90      0.90      0.90      8238



Вышло лучше базовой логистической регрессии (правда там мы не использовали категориальные данные, но да ладно), теперь нужно поиграться с параметрами, чтобы улучшить наши метрики

In [34]:
# Используем перебор гипер-параметров дерева и получим лучшую модель
scorer = make_scorer(f1_score, pos_label='yes')

param_dist = {
    'max_depth': [3, 5, 7, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10, 20, 50],
    'min_samples_leaf': [1, 2, 5, 10, 20],
    'max_features': ['sqrt', 'log2', None],
    'criterion': ['gini', 'entropy'],
    'class_weight': ['balanced', {'no': 1, 'yes': 2}, {'no': 1, 'yes': 3}, {'no': 1, 'yes': 5}, None]
}

dt = DecisionTreeClassifier(random_state=42)

random_search = RandomizedSearchCV(
    dt,
    param_distributions=param_dist,
    n_iter=50,
    scoring=scorer,
    cv=3,
    n_jobs=-1,
    random_state=42,
    refit=True
)

random_search.fit(X_train_f, y_train_f)

best_tree = random_search.best_estimator_
y_pred_best = best_tree.predict(X_test_f)

print(classification_report(y_test_f, y_pred_best))

              precision    recall  f1-score   support

          no       0.97      0.93      0.95      7310
         yes       0.59      0.76      0.66       928

    accuracy                           0.91      8238
   macro avg       0.78      0.85      0.81      8238
weighted avg       0.93      0.91      0.92      8238



Здесь мы использовали авто-инструменты, не смотрели на картину логически, это позволило нам не допустить критических ошибок, но и закрыло нам глаза на идеальное решение, которое, возможно, sklearn упустил. Но несмотря на это у нас вышла хорошая модель, которая даже лучше работает, нежели логистическая регрессия (модель даёт хорошие цифры - радостно, соседняя модель такие цифры не накопил - РРРРААААААДОСТНО).
P.S. Для выявления лучшей модели мы использовали метрику f-score, тк она помогает нам в целом оценивать модель, а не конкретно полноту или точность, тк модель не должна быть плоха в среднем по всем метрикам.

## Вывод (финальный по сравнению 2 разных моделей классификации)
В ходе работы мы обучили 2 разные по типу модели, а так же улучшили их (немного ручками и немного автоматически). Увидели, что для наших данных дерево решений оказалось лучше, тк оно лучше работает с "yes" как по точности, так и по полноте (это шикарно), но при этом она работает НЕ ХУЖЕ с "no" по любым метрикам! Т.е. для данной задачи и для данных данных нам подходит Дерево решений (если выбирать всего из 2 моделей).